In [111]:
from subprocess import Popen, PIPE
from time import sleep
import re
from numpy.random import uniform, choice
import os
from bs4 import BeautifulSoup
from IPython.display import Image, HTML, display, clear_output
from time import strftime
from datetime import datetime
from getpass import getpass
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

In [164]:
class InstaBot:
    
    def __init__(self, user, passw, url = 'https://www.instagram.com/', chrome_path = os.getcwd(), #'/usr/bin',
                 hashtags=[], to_follow=10, to_unfollow=10, headless = True, colab=False):
        self.user = user 
        self.passw = passw
        self.url = url
        self.headless = headless
        self.hashtags = hashtags
        self.driver = self.init_chrome(chrome_path, colab=colab)
        self.log_in()
        self.to_follow = to_follow
        self.to_unfollow = to_unfollow
    
    def init_chrome(self, chrome_path, colab=False, wait=10):
        print('Running Chrome.')
        options = webdriver.chrome.options.Options()
        if colab: options.add_argument('--no-sandbox')
        if colab: options.add_argument('--disable-dev-shm-usage')
        if not self.headless: options.add_argument('--window-size=1200x2900')
        if self.headless: options.add_argument('--headless')
        driver = webdriver.Chrome(executable_path=chrome_path+'/chromedriver', options= options)
        driver.implicitly_wait(wait)
        return driver
      
    def log_in(self):
        self.driver.get(self.url + 'accounts/login')
        sleep(uniform(2,4))
        self.driver.find_element_by_xpath('//input[@name="username"]').send_keys(self.user)
        sleep(uniform(2,4))
        self.driver.find_element_by_xpath('//input[@name="password"]').send_keys(self.passw)
        sleep(uniform(2,4))
        self.driver.find_element_by_xpath("//button/*[text() = 'Iniciar' | text() = 'Log In']").click()    
        print('Logging in.')
        sleep(uniform(2,4))
        
                
    def scroll(self):
        """Scrolls follower listings."""
        self.driver.execute_script("document.querySelector('div[role=dialog] ul').parentNode.scrollBy(0,300)")
    
    def get_users(self, which='followers'):
        print(f'Fetching {which}.')
        date = strftime("%m-%d-%Y")
        
        # Get previous follows & its last modified time, if no more recent than 2 days, fetch new
        previous = sorted([(file, os.stat(file).st_mtime) for file in os.listdir(os.getcwd())
                              if f'{self.user}_{which}' in file], key=lambda x: x[1])
        if (len(previous) > 0) and ((datetime.now().timestamp() - previous[-1][1]) / 60 / 60 < 48):
            with open(previous[-1][0], 'r') as inp:
                follows = inp.read().strip().split(',')
            return follows
            
        self.driver.get(self.url + self.user)
        self.n_follows = int(self.driver.find_element_by_xpath("//a[contains(@href, '{0}')]"
                                                               .format(which)).text.split()[0].replace(',', ''))
        self.driver.find_element_by_xpath("//a[contains(@href, '{0}')]".format(which)).click()
        sleep(uniform(1,2))

        n = 0
        while True:
            self.scroll()
            self.scroll()
            sleep(uniform(0.2,1.2))
            #follows = [e.get_attribute('title') for e in self.driver.find_elements_by_xpath("//*[contains(@class, 'notranslate')]")]
            follows = re.findall('notranslate.+?title="(.+?)"', self.driver.page_source)
            if len(follows) >= self.n_follows:
                with open(f'{os.getcwd()}/{self.user}_{which}_{date}.csv', 'w+') as out:
                    out.write(','.join(follows))
                return follows
            else:
                n += 1
                if n >= self.n_follows/10:
                    raise Exception('Error scrolling for users.')
        
    def follow(self, user):
        """Follows or unfollows user"""
        self.driver.get(self.url + user)
        self.driver.find_element_by_xpath("//button[text()='Seguir'|text()='Follow']").click()
        print(f"Following '{user}'.")
        
    def unfollow(self, user):
        """Follows or unfollows user"""
        self.driver.get(self.url + user)
        self.driver.find_element_by_xpath("//button[text()='Following'|text()='Seguido']").click()
        try:
            sleep(uniform(0,1))
            self.driver.find_elements_by_xpath("//button[text()='Unfollow'|text()='Dejar de seguir']")[1].click()
            print(f"Unfollowed '{user}'.")
        except: pass
    
    def fetch_hashtag_users(self, hashtag, n_users = 10):
        self.driver.get('https://www.instagram.com/explore/tags/'+ hashtag + '/')
        first_thumbnail = self.driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div')
        first_thumbnail.click()
        sleep(uniform(0,2))
        
        users = []
        while True:
            # Like
            self.driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[1]/span[1]/button/span').click()
            print('Liked post :)')
            poster = self.driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/header/div[2]/div[1]/div[1]/h2/a').text
            
            i = 1
            commenters = []
            while True:
                try:
                    commenters.append(self.driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/div[1]/ul'\
                                                               f'/ul[{i}]/li[1]/div/div[1]/div[2]/h3/a').get_attribute('title'))
                    i += 1
                except:
                    break

            users.extend([user for user in commenters + [poster] if user not in self.following])
            sleep(uniform(0,2))
            
            if len(users) >= n_users:
                break
            
            self.driver.find_element_by_link_text('Next').click()
        return users[:n_users]
                          
    def expand(self):
        '''incomplete'''
        self.driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[1]/span[2]/button/span').click()
        comment = 'Genial :)!'
        comment_box = self.driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/article/div[2]/section[3]/div/form/textarea')
        comment_box.send_keys(comment)
        comment_box.send_keys(Keys.ENTER)
        sleep(uniform(20, 40))
                          
    def run(self):
        self.followers = self.get_users('followers')
        self.following = self.get_users('following')
        self.non_reciprocal = list(set(self.following) - set(self.followers))
        print(f"{len(self.followers)} followers.")
        print(f"{len(self.following)} following.")
        print(f"{len(self.non_reciprocal)} non reciprocal.")
        
        self.unfollowed = []
        unfollow = choice(self.non_reciprocal, size=self.to_unfollow, replace=False)
        for user in unfollow:
            self.unfollow(user)
            self.unfollowed.append(user)
            sleep(uniform(2,5))
        print(f"Unfollowed {len(self.unfollowed)} users.")
        self.following = list(set(self.following) - set(self.unfollowed))
            
        self.new_following = []
        for hashtag in self.hashtags:
            print(f"Fetching from '#{hashtag}'")
            new_users = self.fetch_hashtag_users(hashtag, n_users=self.to_follow//len(self.hashtags))
            for user in new_users:
                self.follow(user)
                self.new_following.append(user)
                sleep(uniform(2,8))
        print(f"Following {len(self.new_following)} new users.")
        self.following.extend(self.new_following)

In [167]:
mente = InstaBot(user='ciencia.para.la.mente',
                  passw='mgmgmg77',
                  hashtags=['neurociencias', 'cerebro', 'psicologia', 'autoayuda'], 
                  to_follow = 50,
                  to_unfollow = 2,
                  headless=False
                )

Running Chrome.
Logging in.


In [168]:
mente.run()

Fetching followers.
Fetching following.
8 followers.
15 following.
9 non reciprocal.
Unfollowed 'migimnasiomental'.
Unfollowed 'luisilnoepaz'.
Unfollowed 2 users.
Fetching from '#neurociencias'
Liked post :)
Following 'yeanettedaneau'.
Following 'caro_love86'.
Following 'neuroconducta'.
Following 'marymar20cant'.
Following 'jessicayedraguillen031287'.
Following 'egleearrivillaga'.
Following 'natty_crea_aprende_chile'.
Following 'xarrieterapiasnaturales'.
Following 'yosoymujerfuerte'.
Following 'cidapacheco4933'.
Following 'cesarig19317'.
Following 'neuroconducta'.
Fetching from '#cerebro'
Liked post :)
Following 'dioelismatos59'.
Following 'yayanunez54'.
Following 'yesenia.godinez.779'.
Following 'xiomara_moncada'.
Following 'xiomara_moncada'.
Following 'marino_nora'.
Following 'millionhair2018'.
Following 'nellielizalde'.
Following 'chayo_ayala08'.
Following 'mari_hidalgo60'.
Following 'xunxito_boy'.
Following 'fanny15c'.
Fetching from '#psicologia'
Liked post :)
Following 'bacelarwan